# Ridge, Lasso y Elastic Net

En este notebook aplicamos tres algoritmos de regresión lineal regularizada:

- **Ridge Regression (L2):** penaliza el tamaño de los coeficientes para evitar sobreajuste.  
- **Lasso Regression (L1):** además de penalizar, puede llevar coeficientes a 0, haciendo selección automática de variables.  
- **Elastic Net (L1 + L2):** combina Ridge y Lasso, equilibrando estabilidad y selección de variables.

Usamos el dataset **escalado** porque estos algoritmos son sensibles a la magnitud de las variables.  
Al final compararemos métricas y veremos cómo afecta la regularización al modelo.


## **Paso 1. Importación de librerías**


In [ ]:
# ===================================
# 1. Importar librerías
# ===================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
## **Paso 2. Cargar datasets procesados (escalados)**

En este paso cargamos el dataset procesado y **escalado** desde el notebook `EDA_processed`.  
También separamos los datos en entrenamiento y validación (80/20). 

Por eso cargamos el dataset **escalado** (`features_scaled.csv`), generado en el notebook `EDA_processed.ipynb` con `StandardScaler`.  

Esto asegura que todas las variables tengan media 0 y varianza 1, evitando que variables con unidades más grandes (por ejemplo PIB o gasto sanitario) dominen a otras más pequeñas.


In [ ]:
# ===================================
# 2. Cargar dataset escalado 
# ===================================

X_scaled = pd.read_csv("../data/processed/features_scaled.csv")
y = pd.read_csv("../data/processed/target_y.csv").squeeze()  # convertir a Serie


# División train / validación
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print("Tamaño entrenamiento:", X_train.shape, y_train.shape)
print("Tamaño validación:", X_valid.shape, y_valid.shape)


## **Paso 3. Ridge Regression (L2)**

Ridge añade una penalización L2 a la regresión lineal, reduciendo el tamaño de los coeficientes.  
Esto ayuda a **controlar el sobreajuste** y mejorar la estabilidad del modelo cuando hay multicolinealidad.  
No elimina variables, pero las reduce en magnitud.

Entrenamos el modelo con un valor inicial de `alpha=1.0` y calculamos las métricas:
- **RMSE**: error cuadrático medio (raíz).  
- **MAE**: error absoluto medio.  
- **R²**: proporción de la varianza explicada.


### Interpretación: Ridge Regression

- Ridge ha penalizado los coeficientes más grandes, reduciendo su magnitud.  
- Esto ayuda a controlar el **sobreajuste** y estabiliza el modelo frente a variables correlacionadas.  
- Observando las métricas:  
  - **RMSE bajo** → buenas predicciones en validación.  
  - **MAE** nos da el error promedio en unidades originales (años de esperanza de vida).  
  - **R²** cercano a 1 indica qué porcentaje de la variabilidad se explica por el modelo.  

Si Ridge mejora respecto a la **Regresión Lineal simple**, significa que la regularización está ayudando.


In [ ]:
# ===================================
# 3. Ridge Regression
# ===================================
ridge = Ridge(alpha=1.0, random_state=42)  # alpha controla la regularización
ridge.fit(X_train, y_train)
preds_ridge = ridge.predict(X_valid)

rmse_ridge = np.sqrt(mean_squared_error(y_valid, preds_ridge))
mae_ridge = mean_absolute_error(y_valid, preds_ridge)
r2_ridge = r2_score(y_valid, preds_ridge)

print("📊 Ridge Regression")
print(f"RMSE: {rmse_ridge:.3f}")
print(f"MAE: {mae_ridge:.3f}")
print(f"R²: {r2_ridge:.3f}")

# --- Gráfico: valores reales vs predichos ---
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_valid, y=preds_ridge, alpha=0.5)
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], 'r--')
plt.title("Ridge: Valores reales vs predichos")
plt.xlabel("Valores reales")
plt.ylabel("Predicciones")
plt.show()



## **Paso 4. Lasso Regression (L1)**

Lasso, no solo penaliza como Ridge, sino añade una penalización L1 que fuerza a algunos coeficientes a ser **exactamente 0**.  
Esto significa que realiza **selección automática de variables**, dejando solo las más relevantes.

👉 Entrenamos con `alpha=0.1` y calculamos las métricas.  
Además, podemos observar qué variables han quedado con coeficientes 0.

### Interpretación: Lasso Regression

- Lasso no solo penaliza como Ridge, sino que también **elimina variables irrelevantes** (coeficientes = 0).  
- Esto lo convierte en un método útil para **selección automática de características**.  
- Fíjate en la salida: cuántas variables fueron eliminadas.  

- Si el RMSE o R² no mejoran mucho respecto a Ridge, puede significar que casi todas las variables son útiles.  
- Si mejora y además reduce variables, nos está dando un modelo **más simple y explicativo**.


In [ ]:
# ===================================
# 4. Lasso Regression
# ===================================
lasso = Lasso(alpha=0.01, random_state=42)  # alpha mayor → más coeficientes llevados a cero
lasso.fit(X_train, y_train)
preds_lasso = lasso.predict(X_valid)

rmse_lasso = np.sqrt(mean_squared_error(y_valid, preds_lasso))
mae_lasso = mean_absolute_error(y_valid, preds_lasso)
r2_lasso = r2_score(y_valid, preds_lasso)

print("📊 Lasso Regression")
print(f"RMSE: {rmse_lasso:.3f}")
print(f"MAE: {mae_lasso:.3f}")
print(f"R²: {r2_lasso:.3f}")


# Variables eliminadas por Lasso
coef_lasso = pd.Series(lasso.coef_, index=X_train.columns)
print("\nNúmero de variables eliminadas por Lasso:", (coef_lasso == 0).sum())


# --- Gráfico: valores reales vs predichos ---
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_valid, y=preds_lasso, alpha=0.5, color="orange")
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], 'r--')
plt.title("Lasso: Valores reales vs predichos")
plt.xlabel("Valores reales")
plt.ylabel("Predicciones")
plt.show()




## **Paso 5. Elastic Net (L1 + L2)**

Elastic Net combina las penalizaciones L1 (Lasso) y L2 (Ridge).  
Controla dos hiperparámetros:
- **alpha**: fuerza de la regularización.  
- **l1_ratio**: proporción entre L1 y L2 (ejemplo: 0.5 = mitad Lasso, mitad Ridge).

👉 Entrenamos con `alpha=0.1` y `l1_ratio=0.5` y calculamos las métricas.


### Interpretación: Elastic Net

- Elastic Net combina lo mejor de Ridge y Lasso:  
  - **L2** para estabilizar el modelo frente a multicolinealidad.  
  - **L1** para eliminar variables innecesarias.  
- El hiperparámetro `l1_ratio` define el equilibrio:  
  - 0.0 → Ridge puro.  
  - 1.0 → Lasso puro.  
  - 0.5 → mezcla equilibrada.  

Si Elastic Net logra métricas similares o mejores que Ridge y Lasso, suele ser una **opción más robusta**, ya que se adapta mejor a distintos escenarios.


In [ ]:
# ===================================
# 5. Elastic Net
# ===================================
en = ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=42)  
# l1_ratio=0.5 → combina Ridge (L2) y Lasso (L1) al 50%
en.fit(X_train, y_train)
preds_en = en.predict(X_valid)

rmse_en = np.sqrt(mean_squared_error(y_valid, preds_en))
mae_en = mean_absolute_error(y_valid, preds_en)
r2_en = r2_score(y_valid, preds_en)

print("📊 Elastic Net")
print(f"RMSE: {rmse_en:.3f}")
print(f"MAE: {mae_en:.3f}")
print(f"R²: {r2_en:.3f}")

# --- Gráfico: valores reales vs predichos ---
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_valid, y=preds_en, alpha=0.5, color="green")
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], 'r--')
plt.title("Elastic Net: Valores reales vs predichos")
plt.xlabel("Valores reales")
plt.ylabel("Predicciones")
plt.show()



## **Paso 6. Comparativa visual de métricas entre modelos**

Vamos a comparar los resultados de **Ridge, Lasso y Elastic Net** en un gráfico de barras.  
Esto nos permite ver rápidamente cuál tiene mejor desempeño en RMSE, MAE y R².


### 🔎 Interpretación: Comparativa Ridge vs Lasso vs Elastic Net

- El gráfico muestra los valores de **RMSE, MAE y R²** para los tres algoritmos.  
- Puntos clave a observar:  
  - ¿Qué modelo tiene el **menor RMSE**? → predicciones más precisas.  
  - ¿Qué modelo tiene el **mayor R²**? → mejor capacidad explicativa.  
  - Lasso podría perder algo de precisión si eliminó variables importantes.  
  - Ridge puede ser más estable cuando muchas variables están correlacionadas.  
  - Elastic Net es un término medio que muchas veces ofrece un **balance óptimo**.

👉 Si los tres modelos dan métricas similares, Ridge suele ser suficiente.  
👉 Si hay muchas variables irrelevantes, Lasso o Elastic Net pueden ser mejores.







In [ ]:
# ===================================
# 6. Comparativa de métricas entre modelos
# ===================================
results = pd.DataFrame({
    "Modelo": ["Ridge", "Lasso", "Elastic Net"],
    "RMSE": [
        np.sqrt(mean_squared_error(y_valid, preds_ridge)),
        np.sqrt(mean_squared_error(y_valid, preds_lasso)),
        np.sqrt(mean_squared_error(y_valid, preds_elastic)),
    ],
    "MAE": [
        mean_absolute_error(y_valid, preds_ridge),
        mean_absolute_error(y_valid, preds_lasso),
        mean_absolute_error(y_valid, preds_elastic),
    ],
    "R2": [
        r2_score(y_valid, preds_ridge),
        r2_score(y_valid, preds_lasso),
        r2_score(y_valid, preds_elastic),
    ]
})

plt.figure(figsize=(8,5))
results.set_index("Modelo")[["RMSE","MAE","R2"]].plot(kind="bar", figsize=(10,6))
plt.title("Comparativa de Ridge, Lasso y Elastic Net")
plt.ylabel("Valor")
plt.xticks(rotation=0)
plt.show()

results


## **Nota sobre el ajuste de hiperparámetros**

En este notebook usamos valores fijos de `alpha` y `l1_ratio`, pero podrían ajustarse automáticamente usando **GridSearchCV** o **Optuna**, como hacemos en Random Forest y XGBoost.  

- Para **Ridge y Lasso**, se suele ajustar `alpha`.  
- Para **Elastic Net**, se ajustan tanto `alpha` como `l1_ratio`.  

Esto permitiría encontrar la mejor combinación que minimice RMSE o maximice R².


## **Paso 7 – Optimización de hiperparámetros con GridSearchCV**

Hasta ahora usamos valores fijos de `alpha` (y `l1_ratio` en Elastic Net).  
Sin embargo, estos hiperparámetros pueden influir mucho en el rendimiento del modelo.

👉 Con **GridSearchCV** exploramos diferentes combinaciones de parámetros:  
- Para **Ridge y Lasso** → `alpha` (fuerza de regularización).  
- Para **Elastic Net** → `alpha` y `l1_ratio` (equilibrio entre L1 y L2).  

### Qué esperamos:
- Valores de `alpha` muy bajos ≈ modelo casi igual a regresión lineal.  
- Valores de `alpha` muy altos ≈ modelo demasiado restringido, baja capacidad de predicción.  
- En Elastic Net, ver cómo el balance `l1_ratio` afecta al resultado.  

📊 Mostraremos un heatmap de RMSE promedio para cada modelo y concluiremos qué combinación es más aceptable para nuestro dataset de **Esperanza de Vida**.


In [ ]:
# ===================================
# 7. Optimización con GridSearchCV
# ===================================
from sklearn.model_selection import GridSearchCV

# Definimos el espacio de búsqueda
param_grid_ridge = {"alpha": [0.01, 0.1, 1, 10, 100]}
param_grid_lasso = {"alpha": [0.01, 0.1, 1, 10, 100]}
param_grid_enet = {
    "alpha": [0.01, 0.1, 1, 10],
    "l1_ratio": [0.1, 0.5, 0.9]
}

# Modelos
ridge = Ridge(random_state=42)
lasso = Lasso(random_state=42, max_iter=5000)
elastic = ElasticNet(random_state=42, max_iter=5000)

# GridSearchCV
ridge_search = GridSearchCV(ridge, param_grid_ridge, cv=5,
                            scoring="neg_root_mean_squared_error", n_jobs=-1)
lasso_search = GridSearchCV(lasso, param_grid_lasso, cv=5,
                            scoring="neg_root_mean_squared_error", n_jobs=-1)
enet_search = GridSearchCV(elastic, param_grid_enet, cv=5,
                           scoring="neg_root_mean_squared_error", n_jobs=-1)

# Entrenar búsquedas
ridge_search.fit(X_train, y_train)
lasso_search.fit(X_train, y_train)
enet_search.fit(X_train, y_train)

# Mostrar mejores parámetros
print("📊 Mejores parámetros encontrados:")
print("Ridge:", ridge_search.best_params_, "→ RMSE:", -ridge_search.best_score_)
print("Lasso:", lasso_search.best_params_, "→ RMSE:", -lasso_search.best_score_)
print("Elastic Net:", enet_search.best_params_, "→ RMSE:", -enet_search.best_score_)


### 🔎 Interpretación de resultados

- Cada búsqueda devuelve el valor de `alpha` (y `l1_ratio` en Elastic Net) que **minimiza el RMSE promedio en validación cruzada**.
- Un menor RMSE significa que el modelo generaliza mejor.
- Si:
  - **Ridge** es el ganador → hay multicolinealidad pero todas las variables aportan.  
  - **Lasso** es el ganador → algunas variables son irrelevantes y conviene eliminarlas.  
  - **Elastic Net** gana → necesitamos un balance entre ambos mundos.  

👉 Para este dataset (Esperanza de Vida), donde hay muchas variables correlacionadas pero casi todas con relevancia, **Ridge o Elastic Net suelen ser más adecuados**.  


## **Paso 8 – Heatmap de Elastic Net (α vs l1_ratio)**

Para visualizar cómo afectan los hiperparámetros de **Elastic Net** al rendimiento,
vamos a generar un **heatmap de RMSE promedio** según la combinación de:

- **α (alpha):** controla la fuerza de regularización.  
- **l1_ratio:** equilibrio entre penalización L1 (como Lasso) y L2 (como Ridge).  

### Interpretación esperada:
- **Valores pequeños de α** → modelo se parece a una regresión lineal (menos regularización).  
- **Valores grandes de α** → modelo muy penalizado, peor rendimiento.  
- **l1_ratio cercano a 1** → se comporta más como Lasso (selección de variables).  
- **l1_ratio cercano a 0** → se comporta más como Ridge (todas las variables cuentan).  

📊 El heatmap nos permitirá ver en qué zona (α, l1_ratio) el modelo obtiene menor RMSE (mejor desempeño).


In [ ]:
# ===================================
# 8. Heatmap comparativo de métricas (Ridge, Lasso, Elastic Net)
# ===================================

# Creamos un DataFrame con las métricas
metrics = pd.DataFrame({
    "Ridge": [rmse_ridge, mae_ridge, r2_ridge],
    "Lasso": [rmse_lasso, mae_lasso, r2_lasso],
    "Elastic Net": [rmse_en, mae_en, r2_en]
}, index=["RMSE", "MAE", "R²"])

plt.figure(figsize=(8,5))
sns.heatmap(metrics, annot=True, fmt=".3f", cmap="Blues", cbar=False)
plt.title("Comparación de métricas entre Ridge, Lasso y Elastic Net")
plt.xlabel("Modelo")
plt.ylabel("Métrica")
plt.show()


## **Paso 9 – Conclusiones: Ridge, Lasso y Elastic Net**

Tras entrenar y evaluar los tres modelos de regularización, podemos destacar lo siguiente:

### 🔹 Ridge
- Penaliza los coeficientes grandes con una regularización **L2**.  
- Tiende a **mantener todas las variables**, pero reduciendo sus pesos.  
- Funciona bien cuando muchas variables tienen **efecto pequeño pero distribuido**.  
- En este dataset, Ridge suele mostrar un desempeño estable y reduce el sobreajuste respecto a la regresión lineal simple.

### 🔹 Lasso
- Usa penalización **L1**, que fuerza a algunos coeficientes a ser exactamente **0**.  
- Actúa como un **selector automático de variables**.  
- Puede ser útil en datasets con muchas variables irrelevantes o redundantes.  
- En este caso, Lasso probablemente elimine algunas variables menos influyentes en la predicción de la esperanza de vida.  
- Si el RMSE es mayor que en Ridge, significa que su efecto de selección no aporta mucho valor aquí.

### 🔹 Elastic Net
- Combina **L1 y L2** mediante el hiperparámetro `l1_ratio`.  
- Es flexible: puede comportarse como Ridge (cuando l1_ratio→0) o como Lasso (cuando l1_ratio→1).  
- Es útil cuando hay **muchas variables correlacionadas** y algunas deben ser eliminadas.  
- Con el heatmap vimos en qué zona de parámetros el RMSE es menor: ese equilibrio nos dice qué proporción de Ridge/Lasso es más adecuada.

---

### 📊 Comparación global en este dataset
- **Ridge** → suele dar el mejor equilibrio entre sesgo y varianza.  
- **Lasso** → puede ser menos preciso en RMSE, pero ofrece interpretabilidad (elige pocas variables).  
- **Elastic Net** → permite afinar entre ambos mundos, siendo más robusto si hay multicolinealidad en las variables.

---

✅ Para este dataset de esperanza de vida (con un número moderado de features ya preprocesadas):  
- **Ridge** suele ser la opción más sólida en métricas.  
- **Lasso** puede ser interesante para simplificar el modelo.  
- **Elastic Net** es una buena alternativa si se quiere un balance y explorar la importancia de variables.


## **Paso 10 – Comparación de métricas entre Ridge, Lasso y Elastic Net**

Para comparar de forma clara los tres modelos, recopilamos las métricas principales:

- **RMSE (Root Mean Squared Error):** mide el error promedio en las predicciones, penalizando más los errores grandes.  
- **MAE (Mean Absolute Error):** mide el error promedio absoluto, más robusto frente a outliers.  
- **R² (Coeficiente de determinación):** indica la proporción de varianza explicada por el modelo (más cercano a 1 es mejor).

La siguiente tabla resume los resultados obtenidos.


In [ ]:
# ===================================
# 10. Comparación de métricas entre modelos
# ===================================
results = pd.DataFrame({
    "Modelo": ["Ridge", "Lasso", "Elastic Net"],
    "RMSE": [rmse_ridge, rmse_lasso, rmse_en],
    "MAE": [mae_ridge, mae_lasso, mae_en],
    "R²": [r2_ridge, r2_lasso, r2_en]
})

# Mostrar tabla ordenada por RMSE (menor es mejor)
results = results.sort_values(by="RMSE")
display(results)

# Gráfico comparativo
plt.figure(figsize=(8,5))
sns.barplot(x="Modelo", y="RMSE", data=results, hue="Modelo", legend=False, palette="Blues_r")
plt.title("Comparación de modelos: RMSE")
plt.ylabel("RMSE")
plt.show()


# ===================================
# Guardar resultados en CSV
# ===================================

# ===================================
# Función para guardar métricas estandarizadas
# ===================================
def save_results(model_name, y_true, y_pred, filepath):
    """
    Calcula métricas (RMSE, MAE, R²) y guarda en CSV estandarizado.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    results = pd.DataFrame([{
        "Modelo": model_name,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    }])

    results.to_csv(filepath, index=False)
    print(f"✅ Resultados de {model_name} guardados en {filepath}")
    return results


# ===================================
# Guardar métricas Ridge
# ===================================
results_ridge = save_results(
    "Ridge",
    y_valid,
    preds_ridge,
    "../data/results_ridge.csv"
)


# ===================================
# Guardar métricas Lasso
# ===================================
results_lasso = save_results(
    "Lasso",
    y_valid,
    preds_lasso,
    "../data/results_lasso.csv"
)


# ===================================
# Guardar métricas Elastic Net
# ===================================
results_en = save_results(
    "Elastic Net",
    y_valid,
    preds_en,
    "../data/results_elasticnet.csv"
)


